In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from transformers import UMT5ForConditionalGeneration, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
model_name = "google/umt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="../cache")


In [3]:
model = UMT5ForConditionalGeneration.from_pretrained("results12/checkpoint-9116")

In [4]:
import pandas as pd

df = pd.read_csv("test_without_answers.tsv", sep="\t")


df['natural_sentence'] = df['toxic_sentence']


In [5]:
language_prompts = {
    "en": "translate from English to English: ",
    "ru": "translate from Russian to Russian: ",
    "uk": "translate from Ukrainian to Ukrainian: ",
    "de": "translate from German to German: ",
    "es": "translate from Spanish to Spanish: ",
    "am": "translate from Amharic to Amharic: ",
    "zh": "translate from Chinese to Chinese: ",
    "ar": "translate from Arabic to Arabic: ",
    "hi": "translate from Hindi to Hindi: ",
}

In [6]:
toxics = []
for idx in df.index:
    toxic_sent = language_prompts[df['lang'][idx]] + df['toxic_sentence'][idx]
    toxics.append(toxic_sent)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="../cache",max_length=512,truncation=True,padding='max_length')

In [8]:
from transformers import pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device='cuda',
    batch_size=32
)

neutrals = pipe(toxics)

/home/azubrey/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [9]:
len(neutrals)

5400

In [11]:
neutrals2 = []
for line in neutrals:
    neutrals2.append(line['generated_text'])

In [16]:
neutrals3 = []
j = 0
for i, line in enumerate(neutrals2):
    if line == '':
        neutrals3.append(df['toxic_sentence'][i])
        j = j+1
    else:
        neutrals3.append(line)
print('blank lines: ', j)

blank lines:  8


In [17]:
df2 = pd.read_csv("test_without_answers.tsv", sep="\t")

In [18]:
df2['neutral_sentence'] = neutrals3

In [19]:
df3 = df2[['toxic_sentence','neutral_sentence','lang']]

In [20]:
df3.to_csv("my_submission.tsv", sep="\t", index=False)